# Riemann scheme. 

Lets consider same setup as in [ex_3a](https://github.com/AST-Course/AST5110/blob/main/ex_3a.ipynb) Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$ 

for the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$

whereby $A = 0.02$ , $x_c = 0.70$, $W = 0.1$. Let the solution evolve until time $t_f = 100$. However, let's now implement a new time-step method. Find the conservative form of the equation and implement a Rieman solver [wiki:Rieman Solvers](https://github.com/AST-Course/AST5110/wiki/Riemann-solvers). 

Add this to your library and solve the previous simulation imposing the CFL condition. For this exercise, fill in `nm_lib` the function `evolv_Rie_uadv_burgers`. 

#### 1 step

Compute left and right of $u$, i.e., $u_L$ and $u_R$

#### 2 step
Compute from those the flux, i.e., $F_L$ and $F_R$

#### 3 step
Compute the propagating speed ($v_a[i]=max(|u[i],|u[i-1]|)$)

#### 4 step
Compute the interface fluxes (Rusanov)

$rhs=(F_R+F_L)/2 - v_a (U_R -U_L)$

#### 5 step
Advance in time $u^{n+1}=u^n + dt\times rhs$. What is dt? 


Is this method less or more diffusive than Lax method? Find the analytical solution and compare. Where and when is better or worse than the Lax method? 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from importlib import reload

from nm_lib import nm_lib as nm, utils as utils

plt.style.use("fast")

def get_uadv_t0(xx: np.ndarray, A: float = 0.02, W: float = 0.1, xc: float = 0.7) -> np.ndarray:
    tanh_term = np.tanh((xx + xc) / W) - np.tanh((xx - xc) / W)
    return A * tanh_term

def get_u_exact(xx, t, uu):
    y = utils.get_periodic_value(xx[0], xx[-1], xx - get_uadv_t0(xx)*t)
    # print(y)
    # print(xx-uu*t)
    return get_uadv_t0(y)

if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 128*4
    tf = 300

    xx, _ = utils.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx)

In [ ]:
if __name__ == "__main__":
    reload(nm)
    sol_dict = {}
    sol_dict["Rie"] = (tt_Rie, uunt_Rie) = nm.evolv_Rie_uadv_burgers(xx, uut0, cfl_cut=0.98, end_time=tf)
    sol_dict["Lax"] = (tt_Lax, uunt_Lax) = nm.evolv_Lax_uadv_burgers(xx, uut0, cfl_cut=0.98, end_time=tf)
    
    display(HTML(utils.animate_us(sol_dict, tt_Rie[::30], xx)))
    plt.close()

<span style="color:Orange">

### Exact solution below not working yet

In [ ]:
if __name__ == "__main__":
    uu_exact = np.zeros((len(tt_Rie), nint+1))
    uu_exact[0,:] = uut0
    for n in range(len(tt_Rie)-1):
        uu_exact[n+1] = get_u_exact(xx, tt_Rie[n], uu_exact[n])
    display(HTML(utils.animate_u(tt_Rie[::10], uu_exact[::10], xx)))

---
## TVD scheme. 

Combine the Lax method with the Rieman solver using a flux limiter scheme. For this, one needs to identify how large are the gradients. Lets define: 

$$r^{+}_i = \frac{u_i-u_{i-1}}{u_{i+1}+u_i}$$

What kind of properties do you see on $r^{+}_i$? e.g., what happens when $r\ge0$? or $r\le0$? 

now lets consider the following flux limiter: 

$$\phi = max\left(0,min\left(\theta r,\frac{1+r}{2},\theta \right)\right),\, \theta =[1,2]$$

Finally, combine the Lax from previous excersice with the Riemann solver using the flux limiter as follows: 

$$u^{n+1}_i = u^n_i + dt (f^n_{i+1/2}-f^n_{i-1/2})$$

Where 

$$f^n_{i+1/2} = f^{Riemann}_{i+1/2} + \phi^n_i (f^{L}_{i+1/2}-f^{Riemann}_{i+1/2})$$



When becomes purely Rieman solver? And Lax? Which others methods you can find depending on what is $r$ of $\phi$. 